In [21]:
import pandas as pd
import numpy as np
import pickle
import sys
sys.path.append('../d2c')
from d2c import D2C as D2C_
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score, f1_score

from imblearn.ensemble import BalancedRandomForestClassifier

In [22]:

df = pd.read_csv('../data/deleteme_descriptors.csv')
#flattening
# df = df[(df['edge_dest'] < 3) & (df['edge_source'] > 2)].sort_values(by=['graph_id','edge_source', 'edge_dest']).reset_index(drop=True)

In [24]:
df.is_causal.value_counts()

0    1000
1      56
Name: is_causal, dtype: int64

In [14]:
#suppress fture warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [25]:
X = df.drop(['graph_id','edge_source','edge_dest', 'is_causal'], axis=1)
y = df['is_causal']
#train test split 
logo = LeaveOneGroupOut()
groups = df['graph_id']
brf_scores = pd.DataFrame(columns=['accuracy_train','precision_train','recall_train', 'f1_train', 'auc_train', 'accuracy_test', 'precision_test', 'recall_test', 'f1_test', 'auc_test'])
counter = 0
predicted_values = {}
predicted_probabilities = {}
real_values = {}
for train_index, test_index in logo.split(X, y, groups):
    current_group = groups[test_index[0]]
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    brf = BalancedRandomForestClassifier(n_estimators=10, max_depth=10, random_state=0, n_jobs=10)
    brf.fit(X_train, y_train)

    accuracy_train = brf.score(X_train, y_train)
    precision_train = precision_score(y_train, brf.predict(X_train))
    recall_train = recall_score(y_train, brf.predict(X_train))
    f1_train = f1_score(y_train, brf.predict(X_train))
    auc_train = roc_auc_score(y_train, brf.predict_proba(X_train)[:,1])

    y_hat = brf.predict(X_test)
    y_hat_proba = brf.predict_proba(X_test)[:,1]
    

    accuracy_test = brf.score(X_test, y_test)
    precision_test = precision_score(y_test, y_hat)
    recall_test = recall_score(y_test, y_hat)
    f1_test = f1_score(y_test, y_hat)
    auc_test = roc_auc_score(y_test, y_hat_proba)

    predicted_values[len(brf_scores)] = y_hat
    predicted_probabilities[len(brf_scores)] = y_hat_proba
    real_values[len(brf_scores)] = y_test


    brf_scores.loc[len(brf_scores)] = [accuracy_train, precision_train, recall_train, f1_train, auc_train, accuracy_test, precision_test, recall_test, f1_test, auc_test]


    counter += 1
    print(counter)
    if counter == 20:
        break


1
2
3
4
5
6
7
8


In [26]:
brf_scores

,accuracy_train,precision_train,recall_train,f1_train,auc_train,accuracy_test,precision_test,recall_test,f1_test,auc_test
0,0.632035,0.124675,0.941176,0.220183,0.908542,0.537879,0.017241,0.200000,0.031746,0.428346
1,0.627706,0.117949,1.000000,0.211009,0.934349,0.522727,0.050847,0.300000,0.086957,0.360656
2,0.600649,0.122010,0.962264,0.216561,0.920542,0.454545,0.000000,0.000000,0.000000,0.260982
3,0.650433,0.118132,0.955556,0.210269,0.912805,0.568182,0.074074,0.363636,0.123077,0.388054
4,0.601732,0.107579,0.936170,0.192982,0.898045,0.515152,0.089552,0.666667,0.157895,0.496387
5,0.571429,0.112867,0.943396,0.201613,0.888222,0.659091,0.022727,0.333333,0.042553,0.633075
6,0.490260,0.090909,0.979167,0.166372,0.913230,0.409091,0.051282,0.500000,0.093023,0.410282
7,0.588745,0.112412,0.979592,0.201681,0.926939,0.583333,0.038462,0.285714,0.067797,0.429143


In [12]:
brf_scores

,accuracy_train,precision_train,recall_train,f1_train,auc_train,accuracy_test,precision_test,recall_test,f1_test,auc_test
0,0.660012,0.824789,0.588223,0.686703,0.765943,0.555556,0.692308,0.529412,0.600000,0.488235
1,0.644602,0.811434,0.571873,0.670910,0.749656,0.370370,0.461538,0.375000,0.413793,0.380682
2,0.658436,0.817084,0.593683,0.687695,0.758713,0.555556,0.631579,0.705882,0.666667,0.452941
3,0.664565,0.820692,0.601977,0.694522,0.766502,0.555556,0.666667,0.588235,0.625000,0.605882
4,0.644602,0.817301,0.565407,0.668409,0.752105,0.481481,0.555556,0.333333,0.416667,0.450000
5,0.652220,0.817695,0.580159,0.678745,0.754947,0.814815,0.944444,0.809524,0.871795,0.841270
6,0.648542,0.817662,0.572821,0.673685,0.753081,0.481481,0.777778,0.368421,0.500000,0.414474
7,0.649899,0.813676,0.579952,0.677215,0.757277,0.666667,0.928571,0.619048,0.742857,0.849206
8,0.638911,0.809804,0.561830,0.663402,0.743900,0.481481,0.692308,0.473684,0.562500,0.552632
9,0.647010,0.812506,0.575605,0.673840,0.746880,0.555556,0.625000,0.625000,0.625000,0.454545
